# `Weekly Market Report Generator`

## In this workbook, I will recreate the `Weekly Market Journal` assignment I went through in MSBX 5225 Advanced Portfolio Management. I intend to report on the following assets:
> **Equities**:
> - S&P 500 (^GSPC)
> - Dow Jones Industrial (^DJIA)
> - Nasdaq (^***)
> - Vanguard Total U.S. Stock Market (VTI)
> - Large Blend Global ex-U.S. ETF (VEU)
> - Five biggest gains/losses ??
> - Energy Index (VDE)

> **Volatility**
> - VIX

> **Fixed Income**:
> - U.S. Yield Curve
> - Corporate Yield Curve

> **Front Month Futures**:
> - Gold (GC=F)
> - Oil (CL=F)
> - Corn (ZC=F)

> **Currencies**:
> - Dollar to Euro (USD-EUR)
> - Dollar to Pound (USD-GBP)
> - Dollar to Yuan (USD-CNY)
> - Bitcoin (BTC-USD)


## Currently, I plan to include the following charts and info for each asset:
> - Simple box-whisker plot for returns over past full week (Last Friday's Close to This Friday's Close)
>> These will be set up to include 52 weeks of data, but initially focused on the past 7 days of prices
>>> Debating if volume or MA's should be included
> - A table with summary stats, such as,
>> Open, Close, Max, Min, and these stats from last week
> - Links to the top news articles from the past week
>> Will need to figure out how to do this. Current thought is simply grab the top headline from Bloomberg/WSJ/Reuters

## To create this weekly report, I will utilize the `EquityAnalysis` script I wrote to collect asset prices/yields, and the `plotly` package to create the deliverable. I will publish the deliverable using the `datapane`API. 
> In order to get after-hours prices (i.e. the weekend prices), I may need to modify that script to collect prices/yields using `AlphaVantage`

## Import the necessary modules

In [ ]:
import datetime as dt
from datetime import date
import pandas as pd
import numpy as np
import re
import EquityAnalysis as EA
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import my_weekly_articles as mwa

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import datapane as dp

dp.login(token="")

## Define useful functions for converting dates from strings to datetime objects, and back again in the desired format for this report

In [ ]:
def my_str_to_date(str):
    return dt.datetime.strptime(str, "%a %d-%m-%y").date()

def my_date_to_str(date, title_format=[False, False]):
    if title_format[0]:
        if title_format[1]:
            return dt.datetime.strftime(date, "%a %d-%B-%y (Week: %U)")
        else:
            return dt.datetime.strftime(date, "%a %d-%B-%y")
            
    else:
        return dt.datetime.strftime(date, "%a %d-%m-%y")

## Define the `EndOfWeek` function that will return the final weekday of the current week

In [ ]:
def EndOfWeek(include_monday=False):
    today = date.today()
    wkday = today.weekday()

    days_til = 4 - wkday
    end      = dt.date(today.year, today.month, today.day + days_til).strftime("%a %d-%m-%y")

    if include_monday:
        start    = dt.date(today.year, today.month, today.day - wkday).strftime("%a %d-%m-%y")
        return [start, end]
    else:
        return [end]

In [ ]:
# Create datetime variables for monday and friday of this week
monday = my_str_to_date(EndOfWeek(True)[0])
friday = my_str_to_date(EndOfWeek(True)[1])    

In [ ]:
dt.date.today() < friday

In [ ]:
def FirstTradingDay(year=dt.date.today().year):
    day1 = dt.date(year-1, 12, 31)  # Start at day 2 since no trading on Jan 1
    
    # Use While Loop to make sure first day is not a weekend
    while day1.weekday() > 4:
        day1 += dt.timedelta(days=-1)
    
    return day1

FirstTradingDay()

## Collect the necessary price/yield info

In [ ]:
Tickers = ['^GSPC', '^DJI', '^IXIC', 'VTI', 'VEU', 'VDE',
           '^VIX',
           'GC=F', 'CL=F', 'ZC=F',
           'EURUSD=X', 'GBPUSD=X', 'CNYUSD=X', 'BTC-USD'
           ]
AsstCls = ['EQUITY' for tick in Tickers]

names   = {'^GSPC': 'S&P 500 Index',
           '^DJI': 'Dow Jones Industrial Average',
           '^IXIC': 'NASDAQ Composite Index',
           'VTI': 'Vanguard Total U.S. Stock Market Index Fund',
           'VEU': 'Vanguard All-World ex-U.S. Large Blend ETF',
           'VDE': 'Vanguard Energy Index Fund ETF',
           '^VIX': 'S&P 500 Volatility Index',
           'GC=F': 'Gold Front Month Futures',
           'CL=F': 'Crude Oil WTI Front Month Futures',
           'ZC=F': 'Corn Front Month Futures',
           'EURUSD=X': 'EURO-USD',
           'GBPUSD=X': 'GBP-USD',
           'CNYUSD=X': 'CNY-USD',
           'BTC-USD': 'Bitcoin-USD'
           }
data = EA.Data(tickers = Tickers,
               period='2y',
               interval='1d')

In [ ]:
data.Collect(DataType='prices')
price_data = data.PriceData

In [ ]:
colors = px.colors.diverging.curl
#colors = colors[:8]
#colors, len(colors)

In [ ]:
price_data

## Collect relevant article links using the `my_weekly_articles.py` script

In [ ]:
article_dfs_lst = []
for asset_class, tick in zip(AsstCls + ['EQUITY'], Tickers + ['YIELD']):
    si = mwa.score_index(asset_class, tick)
    si.go()
    
    index_df = si.index_df
    index_df = index_df.loc[:10, ['LMcD_Neg_Terms',  'LMcD_Pos_Terms',  'LMcD_Tot_Terms',
                                   'Date_Relevance',  'Title_Relevance', 'Relevancy_Score',
                                   'Source', 'Date', 'Title', 'Link']]
    
    ## Create column for plotly/html formatted title-link combo
    index_df.loc[:, 'Article'] = index_df.apply(
        lambda x: f'<a href="{x.Link}">{x.Title}</a>',
        axis=1
    )
    
    article_dfs_lst.append([tick, index_df])
    
    print(f"Collected articles for {tick}")

## Define `plotly_article_table` to create a pandas df that will be used in the final visual

In [ ]:
def plotly_article_table(tick, art_lst):
    # Collect the appropiate df and columns
    for i in art_lst:
        if i[0] == tick:
            df = i[1][['Source', 'Date', 'Article']]
    
    # Clean up the Source and Date columns
    def clean_source(source):
        source = re.sub('seeking_alpha', 'Seeking Alpha', source)
        source = re.sub('MarketWatch', 'Market Watch', source)
        source = re.sub('Associated', 'Assoc.', source)
        source = re.sub('New York Times', 'NYT', source)
        source = re.sub('Wall Street Journal', 'WSJ', source)
        source = re.sub('Marketscreener', 'Market Screener', source)
        source = re.sub('.com|The', '', source)
        source = source.strip()
        
        return source
    df.loc[:, 'Source'] = df.Source.apply(lambda x: f"<b>{clean_source(x)}</b>")
    df.loc[:, 'Date']   = df.Date.apply(lambda x: my_date_to_str(x, [True, False]))
    return df

plotly_article_table('YIELD', article_dfs_lst)

## Define `SumDF` to create a summary table of return data

In [ ]:
def SumDF(df, tick):
    # Define a storage dictionary
    stats_dict = {}
    for key in ['1wk', '2wk', '1mo', '3mo', '6mo', '1y', '1.5y', '2y']:
        stats_dict[key] = []
    
    # Subset out the Closing prices
    idx = pd.IndexSlice
    df = df.loc[:, idx[['Close', 'PCT_Change'], tick]]
    df.columns = ['Close', '1 Day Change']
    
    # Identify the date of Monday for this week
    monday = my_str_to_date(EndOfWeek(True)[0])
    friday = my_str_to_date(EndOfWeek(True)[1])    
    
    # Calculate the return for this week (based on close of last week)
    deciles = [0, 0.25, 0.5, 0.75, 1]
    decile_data = []
    
    # Start with YTD stats
    temp_df = df.loc[(df.index.date >= FirstTradingDay()) & (df.index.date <= friday), :]
    decile_data.append(list(temp_df.Close))

    ret = round(100 *(temp_df.Close.iloc[-1] / temp_df.Close.iloc[0] - 1), 2)
    chg = round(temp_df.Close.iloc[-1] - temp_df.Close.iloc[0], 2)
    ## Error handling for when yfinance data doesnt provide enough price data
    if str(ret) == 'nan':

        if (str(temp_df.Close.iloc[0]) != 'nan') & (str(temp_df.Close.iloc[-1]) == 'nan'):
            ret = round(100 *(temp_df.Close.iloc[-2] / temp_df.Close.iloc[0] - 1), 2)
            chg = round(temp_df.Close.iloc[-2] - temp_df.Close.iloc[0], 2)

        elif (str(temp_df.Close.iloc[0]) == 'nan') & (str(temp_df.Close.iloc[-1]) != 'nan'):
            ret = round(100 *(temp_df.Close.iloc[-1] / temp_df.Close.iloc[1] - 1), 2)
            chg = round(temp_df.Close.iloc[-1] - temp_df.Close.iloc[1], 2)

        elif (str(temp_df.Close.iloc[0]) == 'nan') & (str(temp_df.Close.iloc[-1]) == 'nan'):
            ret = round(100 *(temp_df.Close.iloc[-2] / temp_df.Close.iloc[1] - 1), 2)
            chg = round(temp_df.Close.iloc[-2] - temp_df.Close.iloc[1], 2)

            i = 0
            while str(ret) == 'nan':
                i += 1
                ret = round(100 *(temp_df.Close.iloc[-1 - i] / temp_df.Close.iloc[0 + i] - 1), 2)
                chg = round(temp_df.Close.iloc[-1 - i] - temp_df.Close.iloc[0 + i], 2)

    mu  = round(temp_df['1 Day Change'].mean(), 2)
    sd  = round(temp_df['1 Day Change'].std(), 2)
    sum_stats = [chg, ret, mu, sd]

    dec_stats = [round(temp_df.Close.quantile(dec),2) for dec in deciles]
    stats_dict['YTD'] = sum_stats + dec_stats
    
    for key, day in zip(list(stats_dict.keys()),
                        [7, 7*2, 7*4, 7*12, 7*24, 7*52, 7*76, 7*104]):
        start_friday = friday + dt.timedelta(days=-day)
        
        ## Use while loop to move `start_friday` back if its a weekend
        while (start_friday.weekday() > 4) | (str(df.loc[df.index.date >= start_friday, 'Close'].iloc[0]) == 'nan'):
            start_friday += dt.timedelta(days=-1)
        
        temp_df = df.loc[(df.index.date >= start_friday) & (df.index.date <= friday), :]
        decile_data.append(list(temp_df.Close))
        
        ret = round(100 *(temp_df.Close.iloc[-1] / temp_df.Close.iloc[0] - 1), 2)
        chg = round(temp_df.Close.iloc[-1] - temp_df.Close.iloc[0], 2)
        ## Error handling for when yfinance data doesnt provide enough price data
        if str(ret) == 'nan':
            
            if (str(temp_df.Close.iloc[0]) != 'nan') & (str(temp_df.Close.iloc[-1]) == 'nan'):
                ret = round(100 *(temp_df.Close.iloc[-2] / temp_df.Close.iloc[0] - 1), 2)
                chg = round(temp_df.Close.iloc[-2] - temp_df.Close.iloc[0], 2)
                
            elif (str(temp_df.Close.iloc[0]) == 'nan') & (str(temp_df.Close.iloc[-1]) != 'nan'):
                ret = round(100 *(temp_df.Close.iloc[-1] / temp_df.Close.iloc[1] - 1), 2)
                chg = round(temp_df.Close.iloc[-1] - temp_df.Close.iloc[1], 2)
                
            elif (str(temp_df.Close.iloc[0]) == 'nan') & (str(temp_df.Close.iloc[-1]) == 'nan'):
                ret = round(100 *(temp_df.Close.iloc[-2] / temp_df.Close.iloc[1] - 1), 2)
                chg = round(temp_df.Close.iloc[-2] - temp_df.Close.iloc[1], 2)

                i = 0
                while str(ret) == 'nan':
                    i += 1
                    ret = round(100 *(temp_df.Close.iloc[-1 - i] / temp_df.Close.iloc[0 + i] - 1), 2)
                    chg = round(temp_df.Close.iloc[-1 - i] - temp_df.Close.iloc[0 + i], 2)
            
        mu  = round(temp_df['1 Day Change'].mean(), 2)
        sd  = round(temp_df['1 Day Change'].std(), 2)
        sum_stats = [chg, ret, mu, sd]
        
        dec_stats = [round(temp_df.Close.quantile(dec),2) for dec in deciles]
        stats_dict[key] = sum_stats + dec_stats
        
    sum_df = pd.DataFrame.from_dict(stats_dict)
    labels = ['HPR($)', 'HPR(%)', 'µ(%)', 'σ(%)', 'Min. Price'] + [f"{int(100 * i)}th PCTL" for i in deciles[1:-1]] + ['Max. Price']
    sum_df.index = labels
    sum_df = sum_df[['YTD'] + [col for col in sum_df.columns if col != 'YTD']]
    
    
    return sum_df, decile_data

stats_df, decile_data = SumDF(price_data, '^GSPC')
stats_df

In [ ]:
stats_df.iloc[:3,:]

## Define `yield_curve` function for collecting US Yield Curve data

In [ ]:
def yield_curve(friday):

    # Collect data provided by the US treasury
    y1_link = f"https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year={friday.year}"
    y2_link = f"https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year={friday.year - 1}"
    y3_link = f"https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year={friday.year - 2}"

    y1_df = pd.read_html(y1_link)[1]
    y2_df = pd.read_html(y2_link)[1]
    y3_df = pd.read_html(y3_link)[1]

    # Concat the data frames
    df = pd.concat([y1_df, y2_df, y3_df], axis=0)

    # Clean up the date column
    df.Date = pd.to_datetime(df.Date, format="%m/%d/%y")
    df.sort_values(by='Date', ascending=True, inplace=True)
    df.reset_index(inplace=True, drop=True)

    # Create list of dates to collect
    my_dates = [friday] + [friday + dt.timedelta(days=-i) for i in [7, 7*2, 7*4, 7*8, 7*12, 7*24, 7*52, 7*76, 7*104]]
    my_keys  = ['This Week', '1wk Ago', '2wk Ago', '1mo Ago', '2mo Ago', '3mo Ago', '6mo Ago', '1y Ago', '1.5y Ago', '2y Ago']
    my_keys  = [my_keys[-i] for i in range(1, len(my_keys)+1)]
    
    # Clean the list of dates
    i = 0
    for date in my_dates:
        i += 1
        while date not in list(df.Date):
            date += dt.timedelta(days=-1)
            my_dates[i-1] = date

    # Concat the dates with relevant keys
    my_keys = [j + " | " + str(i.date()) for i, j in zip(list(df.Date), my_keys)]
    df = df[df.Date.isin(my_dates)]
    #df.Date = my_keys
    df.reset_index(inplace=True, drop=True)
    
    return df

yield_curve(friday)

## Define `Equity_Plot` function to separate out price data for selected equities and create the candlestick plot

In [ ]:
def Equity_Plot(all_prices, tick, art_lst, names, fig, row, col=2):
    idx = pd.IndexSlice
    # Seperate out the needed prices for the desired asset
    prices = all_prices.loc[:, idx[['Open', 'Close', 'Adj Close', 'High', 'Low'], tick]]
    prices.columns = [col[0] for col in prices.columns]
    stats_df, decile_data = SumDF(all_prices, tick)
    
    # Collect the appropiate colors for the box plots
    colors = px.colors.diverging.curl
    colors = colors[:4] + colors[-5:]
    
    # Collect the articles table
    articles = plotly_article_table(tick, art_lst)
        
    # Add the summary box plots for price quartiles
    for i in range(len(decile_data)-1,-1,-1):
        fig.add_trace(go.Box(y=decile_data[i],
                             name=stats_df.columns[i],
                             showlegend=False,
                             marker_color = colors[i]
                            ),
                      row=row,
                      col=1)
        fig.update_xaxes(title="Holding Period",
                         row=row,
                         col=1)
        fig.update_yaxes(title="<b>Price",
                         row=row,
                         col=1)
    
    ## Add the price candlesticks
    fig.add_trace(
        go.Ohlc(
            x     = prices.index,
            open  = prices['Open'],
            high  = prices['High'],
            low   = prices['Low'],
            close = prices['Close'],
            showlegend = False,
            name  = tick,
        ),
        row = row,
        col = 2,
        secondary_y = False,
    )
    
    ## Add the moving averages
    ema12 = prices['Close'].ewm(span=12, adjust=False).mean().round(4)
    ema26 = prices['Close'].ewm(span=26, adjust=False).mean().round(4)
    macd = round(ema12 - ema26, 4)
    signal = macd.ewm(span=9, adjust=False).mean().round(4)
    
    for dat, name, color in zip([ema12, ema26], ['Moving Avg 12', 'Moving Avg 26'], ['royalblue', 'firebrick']):
        fig.add_trace(go.Scatter(x=prices.index,
                                 y=dat,
                                 showlegend = False,
                                 name=name,
                                 line=dict(color=color, width=2)),
                      row=row,
                      col=col)
    
    ## Add MACD to secondary y
    fig.add_trace(go.Bar(x=prices.index,
                         y=macd,
                         showlegend=False,
                         name='MACD',
                         opacity=0.5),
                  row=row,
                  col=col,
                  secondary_y=True)

    fig.add_trace(go.Scatter(x=prices.index,
                     y=signal,
                     mode = 'lines',
                     showlegend=False,
                     name='MACD Signal',
                     opacity=0.5),
              row=row,
              col=col,
              secondary_y=True)

#    fig.update_yaxes(title = "<b>Price",
#                     row=row,
#                     col=col,
#                     secondary_y=False)

    fig.update_yaxes(showgrid=False,
                     title = "<b>MACD",
                     row=row,
                     col=col,
                     title_standoff = 0,
                     secondary_y=True)
    
    fig.update_xaxes(range=(str(date.today() + dt.timedelta(days=-21)),
                            str(date.today() + dt.timedelta(days=1))),
                     rangeslider_thickness=0.0089,
                     row = row,
                     col = col)
    
    ## Add the table for HPR return data
    hpr_dat = stats_df.iloc[:4,:].copy().T
    
    # Reorder cols
#    cols = hpr_dat.columns
#    cols = [cols[i] for i in range(len(cols)-1,-1,-1)]
#    hpr_dat = hpr_dat[cols].T
    
    hpr_dat.reset_index(inplace=True, drop=False)
    hpr_dat.rename(columns={'index': ''}, inplace=True)

    # Create a list of fill colors for the table cells
    cell_fill_color = ['dodgerblue'] + ['ivory'] * 9
    head_fill_color = ['midnightblue'] * 10

    # Create the table object to be added to the figure
    table = go.Table(header = dict(values     = list(hpr_dat.columns),
                                   font       = dict(size=12, color='white'),
                                   line_color = 'darkslategray',
                                   fill_color = head_fill_color,
                                   align      = 'right'),
                     cells  = dict(values     = [hpr_dat[col] for col in hpr_dat.columns],
                                   font       = dict(size=11.5, color='black'),
                                   line_color = 'darkslategray',
                                   fill_color = cell_fill_color,
                                   align      = 'right',
                                   height     = 25),
                     columnwidth = [0.7, 1.15, 1, 0.9, 0.9],
                     )
    fig.add_trace(table, row = row + 3, col = 1)
    
    
    ## Add table for relevant articles
    # Create a list of fill colors for the table cells
    cell_fill_color = ['whitesmoke'] * 3
    head_fill_color = ['palegoldenrod'] * 3

    # Create the table object to be added to the figure
    table = go.Table(header = dict(values     = list(articles.columns),
                                   font       = dict(size=14, color='black'),
                                   line_color = None,
                                   fill_color = head_fill_color,
                                   align      = 'left'),
                     cells  = dict(values     = [articles[col] for col in articles.columns],
                                   font       = dict(size=11.5, color='black'),
                                   line_color = None,
                                   fill_color = cell_fill_color,
                                   align      = 'left',
                                   height     = 25),
                     columnwidth = [1, 1, 1.6],
                     )    
    
    fig.add_trace(table, row = row, col = 3)
    
    
    return prices, fig

## Define `Yield_Plot` which will create the Yield Curve figure

In [ ]:
colors

In [ ]:
def Yield_Plot(yield_df, art_lst, fig, row, col=1):
    
    # Collect the appropiate colors for the box plots
    colors = px.colors.diverging.curl
    colors = colors[:4] + colors[-5:]
    
    # Add the 3D surface plot
    x_dat = yield_df.columns[1:]
    y_dat = yield_df.iloc[:, 0]
    z_dat = yield_df.iloc[:, 1:]
    
    curve = go.Surface(x=x_dat,
                       y=y_dat,
                       z=z_dat,
                       #name=y_dat,
                       colorscale='RdBu',
                       showscale=False,
                       contours = {
                           "x": {"show": True}, # "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
                           "y": {"show": True},
                           "z": {"show": True}  #, "start": 0.5, "end": 1.8, "size": 0.05}
                       }
                      )

    fig.add_trace(curve, row=row, col=1)
    fig.update_traces(row=row, col=1,
                      contours_z = dict(show=True, usecolormap=True,
                                        highlightcolor="limegreen", project_z=True),
                    )
    fig.update_layout(
                      scene = {"xaxis": {"title": "Tenor", "nticks": 20},
                               "yaxis": {"title": "Closing Date"},
                               "zaxis": {"title": "Closing Yield", "nticks": 10}, 
#                               'camera_eye': {"x": 1, "y": 1, "z": 1}, 
                               "aspectratio": {"x": 0.8, "y": 1, "z": 0.5}}
                     )
    
    fig.update_yaxes(title="<b>Yield (%)", row=row, col=col)
    fig.update_xaxes(title="<b>Tenor", row=row, col=col)

    # Collect the articles table
    articles = plotly_article_table('YIELD', art_lst)
            
    ## Add the articles table
    # Create a list of fill colors for the table cells
    cell_fill_color = ['whitesmoke'] * 3
    head_fill_color = ['palegoldenrod'] * 3

    # Create the table object to be added to the figure
    table = go.Table(header = dict(values     = list(articles.columns),
                                   font       = dict(size=14, color='black'),
                                   line_color = None,
                                   fill_color = head_fill_color,
                                   align      = 'left'),
                     cells  = dict(values     = [articles[col] for col in articles.columns],
                                   font       = dict(size=12, color='black'),
                                   line_color = None,
                                   fill_color = cell_fill_color,
                                   align      = 'left',
                                   height     = 25),
                     columnwidth = [1, 1, 1.6],
                     )    
    
    fig.add_trace(table, row = row, col = 3)
    
    return fig

CSdict  = {'type': 'surface', 'is_3d': True, 'colspan': 2, 'rowspan': 5}  # Candlestick plot specs
TBdict2  = {"type": "table", 'colspan': 1, 'rowspan': 5}                   # Table specs

specs = []
for tick in range(0,1):
    specs.append([CSdict, None, TBdict2])
    specs.append([None, None, None])
    specs.append([None, None, None])
    specs.append([None, None, None])
    specs.append([None, None, None])

temp = make_subplots(rows = 5, cols=3, specs = specs)
temp = Yield_Plot(yield_curve(friday), article_dfs_lst, temp, 1)
temp.update_layout(width = 1500, height = 650, hovermode = 'x')
temp.show()

## Define `Add_RangeSelector` function to add a rangeselector to the plotly figures

def Add_RangeSelector(fig):
    ## Add range slider and adjust general plot layout elements
    fig.update_layout(
        xaxis_showticklabels=True,
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=7, label="1wk", step="day", stepmode="backward"),
                    dict(count=14, label="2wk", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=2, label="2m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(count=18, label="1.5y", step="month", stepmode="backward"),
                    dict(step="all")
                ]),
                font=dict(color='black')
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        ),
    )
    return fig

## Create the plots

In [ ]:
CSdict  = {"type": "xy", 'colspan': 1, 'rowspan': 4, 'secondary_y': True}  # Candlestick plot specs
TBdict1  = {"type": "table", 'colspan': 1, 'rowspan': 2}                   # Table specs
TBdict2  = {"type": "table", 'colspan': 1, 'rowspan': 5}                   # Table specs
BPdict  = {"type": "xy", 'colspan': 1, 'rowspan': 3}                       # Boxplot specs
SCdict  = {'type': 'surface', 'is_3d': True, 'colspan': 2, 'rowspan': 5}   # Surface plot specs

specs = []
# Define specs for equity/fi plots
for tick in Tickers:
    specs.append([BPdict, CSdict, TBdict2])
    specs.append([None, None, None])
    specs.append([None, None, None])
    specs.append([TBdict1, None, None])
    specs.append([None, None, None])
# Define specs for yield curve plots
specs.append([SCdict, None, TBdict2])
specs.append([None, None, None])
specs.append([None, None, None])
specs.append([None, None, None])
specs.append([None, None, None])


subtitles_BP = ["Closing Price Quartiles" for tick in Tickers]
subtitles_TB = [["Relevant Articles", "Daily Return Data"] for tick in Tickers]
subtitles_CS = [f"[<i>${tick}</i>] as of {str(min(date.today(), friday))}<br><em>{names[tick]}</em>" for tick in Tickers]

subtitles    = []
# Define subtitles for equity/fi plots
for i in range(len(subtitles_CS)):
    subtitles.append(subtitles_BP[i])
    subtitles.append(subtitles_CS[i])
    subtitles.append(subtitles_TB[i][0])
    subtitles.append(subtitles_TB[i][1])
# Define subtitles for Yield Curve plots
subtitles.append("<em>U.S. Yield Curve")
subtitles.append("Relevant Articles")
    
fig = make_subplots(
    rows = (len(Tickers) + 1) * 5,
    cols = 3,
    column_widths = [1.1, 1.4, 1.1],
    row_heights = [1, 1, 1, 1.25, 1.25] * (len(Tickers) + 1),
    horizontal_spacing = 0.05,
    vertical_spacing = 0.012,
    specs = specs,
    subplot_titles = subtitles,
)

report_dates = [my_date_to_str(friday + dt.timedelta(days=-7), [True, False]), my_date_to_str(friday, [True, False])]
report_title = "Gral's Weekly Market Update"
report_descr = ' to '.join(report_dates)

fig.update_layout(
    title = f"<em>{report_title} | " + report_descr,
    title_font_size = 28,
    template = 'seaborn',
    height = (len(Tickers) + 1) * 650,
    width = 1400,
    autosize = True,
    hovermode = 'x',
)

for i in range(1, len(Tickers) + 1):
    fig['layout'][f'xaxis{i}']['showticklabels'] = True
    
## Add the equity plots
i = 1
for tick in Tickers:
    
    p1, fig = Equity_Plot(price_data, tick, article_dfs_lst, names, fig, row = i)
    i += 5

## Add the yield curve plots
fig = Yield_Plot(yield_df = yield_curve(friday), art_lst = article_dfs_lst, fig = fig, row=i)

#fig = Add_RangeSelector(fig)

fig.show()

## Write up a description to add to the DataPane report

In [ ]:
desc = "\n\nAuthor's Notes:\n\n" + \
       "- These visuals were created using the `Plotly` library in Python with data being collected from `Yahoo Finance`.\n\n" + \
       \
       "- The purpose of this post is to provide a weekly update on important U.S. benchmarks. " + \
       "Primarily, the goal is to provide users a snapshot of what happened over the prior week (i.e. price movements), " + \
       "while also being able to provide some detail as to why (via news aggregation). " + \
       "My inspiration for this project comes from a Master's class I took in the spring of 2021 where we would write " + \
       "weekly 'Market Diary' entries which detailed the weekly price changes on a nominal and percent basis. " + \
       "With the weekly diary entries, we would also provide short written summaries detailing the events of the past week.\n\n" + \
       \
       "- This is currently a rough draft of what I'd like the final deliverable to look like. " + \
       "There are several large items on my ToDo list, including:\n\n" + \
       ">- Add in U.S. Treasury Yield Curve data (with the ability to see how that's changed over the prior few weeks).\n\n" + \
       ">- Include major Fixed Income funds to track those returns as well.\n\n" + \
       ">- Add in weekly news aggregation showing top 5 most relevant articles for each asset from major news outlets. " + \
       "The top news articles will be displayed in a table on the RHS of each assets price data.\n\n" + \
       ">- Add in NLG (Natural Language Generation) functionality which will write a summary of major events over the past week. " + \
       "This will either be on a per-asset basis (ideally) or for all assets detailed in this report.\n\n" + \
       ">- Add in a `RangeSelector` which allows users to expand all candlestick charts to a desired date range. " + \
       "Prior drafts of this report incorporated the `RangeSelector` but it was far too buggy to include in a public draft.\n\n" + \
       \
       ""

desc

## Publish the report

In [ ]:
report_dates = [my_date_to_str(friday + dt.timedelta(days=-7), [True, False]), my_date_to_str(friday, [True, True])]
report_title = "Gral's Weekly Market Update"
report_descr = ' to '.join(report_dates)

# Create report
r = dp.Report(report_title,
              report_descr,
              fig,
              desc,
        )

# Publish the report
r.publish(name=report_title, 
          open=True, 
          visibility=dp.client.api.report.core.Visibility(2),
          description=report_descr)
